In [1]:
#!pip install keplergl
#!pip install geopandas
#!pip install sodapy

from sodapy import Socrata
import geopandas as gpd
from keplergl import KeplerGl

In [2]:
#insert your own token to avoid throttling
#found at https://data.cityofchicago.org/profile
token = None

In [3]:
# creating the date range we'll use for this notebook
start = '2019-01-01'
end = '2020-01-31'

In [4]:
def get_rides(start, end, token=None):
    '''
    Creates a request for the SODA API datasets. 
    
    returns: geopandas coordinates for rides
    '''
    #instantiate the Socrata client
    client = Socrata(
        domain = 'data.cityofchicago.org',
        app_token=token, timeout=100000
    )
    
    #set up the SoSQL call
    select = 'pickup_centroid_location AS geometry, COUNT(trip_id) AS rides'
    group = 'geometry' 
    where = f'trip_start_timestamp between "{start}" and "{end}"'
    
    #make the call for the data, specifying the correct dataset
    results = client.get(
        dataset_identifier='m6dm-c72p',
        select=select, group=group, where=where,
        content_type = 'geojson', limit=100_000
    )
    #close our conneciton
    client.close()
    
    #add data to geopandas
    gdf = gpd.GeoDataFrame.from_features(results).dropna()
    gdf.rides = gdf.rides.astype('int32')
    
    return gdf

In [ ]:
%%time
#calling our data gathering funciton
gdf_point = get_rides(start, end, token)

In [ ]:
defining helper functions to load/save map config files
#(to maintain similar visuals)

def open_config(file_name):
    '''
    helper function to load preconfigured Kepler configs
    '''
    try: 
        current_file = json.load(open(file_name, 'r'))
        print('config loaded')
        return current_file
    except FileNotFoundError:
        print('no config file found. plz initialize map, and use map.config')
        return {}

    
def save_config(config, file_name):
    '''
    helper function to save the current config for Kepler
    '''
    def save():
        with open(file_name, 'w') as f:
                json.dump(config, f)
    try:
        current_file = json.load(open(file_name, 'r'))
        if config != current_file:
            print('changes detected. saving current config')
            save()
        else:
            print('no changes detected')
            
    except FileNotFoundError:
        print('no existing file in directory. saving current config')
        save()

In [ ]:
point_config = open_config('configs/point_config.json')

In [ ]:
map_1 = KeplerGl(height=900, config=point_config)
map_1.add_data(gdf_point, 'rides')
map_1

In [ ]:
save_config(map_1.config, file_name='configs/point_config.json')